## This notebook uses llama-stack-client to do handle the following flow - 

1. Notebook is started when an ansible job log completion data comes in.
1. It examimes if there is any error. If no error, it ends
1. If there is an error:
    - Agent analyzes and recommends
    - Agent opens a jira ticket
    - Agent sends a slack message

It uses client provided tools because running an MCP Server may prove to be an overdose.
It also assumes that the audience need not worry about running a LLM and llama-stack server.

TODO 
- structure the output of the tool
- provide more description etc

In [1]:
import asyncio
import os

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.client_tool import client_tool
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from rich.pretty import pprint
import json
import uuid
from pydantic import BaseModel
from typing import List
#from llama_stack.distribution.library_client import LlamaStackAsLibraryClient

from dotenv import load_dotenv

load_dotenv()
#BRAVE_SEARCH_API_KEY = os.environ["BRAVE_SEARCH_API_KEY"]
HOST=os.environ["HOST"]
PORT=os.environ["LLAMA_STACK_PORT"]
MODEL_NAME=os.environ["INFERENCE_MODEL"]
#TAVILY_SEARCH_API_KEY=os.environ["TAVILY_API_KEY"]

In [2]:
@client_tool
def jiratool(error: str = "error", diagnosis: str = "diagnosis", title: str = "Fishing Expedition") :
    """
    Creates a JIRA Ticket.

    :param error: Pass the Error Message
    :param diagnosis: Pass the suggested Diagnostic
    :param title: Pass in the suggested name of the JIRA Ticket
    :returns: Dictionary containing the JIRALink, Status of the operation and title of the ticket
    """
    dummy_response = """
            JIRA Ticket has been created
    """
    #dummy_response =  {"JIRALink": "http://xx.yy.zz", "Status": "Success", "title":title}
    return dummy_response


In [3]:
@client_tool
def slacktool(channel: str = "foo", message: str = "JIRA Ticket details"):
    """
    Creates a JIRA Ticket.

    :param channel: Slack Channel Name to the send the message
    :param message: JIRA Ticket details
    :returns: Slack Message confirmation
    """
    dummy_response = """
            Slack message has been sent.
    """
    return dummy_response

In [4]:
client = LlamaStackClient(base_url=f"http://{HOST}:{PORT}")
base_agent_config = dict(
    model=MODEL_NAME,
    instructions="You are a helpful assistant.",
    sampling_params={
        "strategy": {"type": "top_p", "temperature": 0.01, "top_p": 0.99},
    },
    tools=[jiratool, slacktool],
)


In [5]:
data_processing_steps = [
    """You can look at the contents of an ansible log and spot the error. 
    You will describe what the error is in a few crisp sentences
    so that a human can take corrective actions.
    
    Input: {input}""".strip()
    ,
    
    """Open a JIRA Ticket
    """,
    
    """Slack the details
    """,
    

]

In [6]:
vanilla_agent_config = {
    **base_agent_config,
}

vanilla_agent = Agent(client, **vanilla_agent_config)
workflow_session_id = vanilla_agent.create_session(session_name=f"vanilla_agent_{uuid.uuid4()}")


In [7]:
def chain(input: str, prompts: List[str]) -> str:
    """Chain multiple LLM calls sequentially, passing results between steps."""

    for i, prompt in enumerate(prompts):    
        response = vanilla_agent.create_turn(
            messages=[
                {
                    "role": "user",
                    "content":f"{prompt}\nInput: {input}"
                }
            ],
        session_id=workflow_session_id,
        stream=False,
    )
        print("========= Turn: ", i, "=========")
        print(response)
        print("========= Agent internal workings=========")
        print(response.output_message.content)
        print("\n")
    
    #return result

In [8]:
def route(input: str) -> str:
    """Route input to specialized prompt using content classification."""
    # First determine appropriate route using LLM with chain-of-thought
    is_there_error = f"""
    You can look at the contents of an ansible log and spot if there is any error. 
    If there is any error you will return Error. 
    If there is no error, you will return Success.
    You are only allowed to return 2 words:
    Error
    Success
    
    Input: {input}""".strip()
    
    route_response = vanilla_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content":f"{is_there_error}\nInput: {input}"
            }
        ],
        session_id=workflow_session_id,
        stream=False,
    )
    
    print("========= Agent internal workings=========")
    print(route_response)
    print("========= Agent internal workings=========")
    print("Observed Status of job:")
    print(route_response.output_message.content)

    return route_response.output_message.content

In [9]:
def flow(input: str,prompts: List[str])-> str:
    route_response = route(input)
    
    if route_response == "Error":
        print("Examining the Error now")
        result = chain(input, prompts)
    else:
        result = "No action Needed"
    return result

In [10]:
#could not connect to the host as the password as expired.
#all tasks successfully finished

input = """
could not connect to the host as the password as expired.
"""
flow(input,data_processing_steps)


========= Agent internal workings=========
Turn(input_messages=[UserMessage(content='You can look at the contents of an ansible log and spot if there is any error. \n    If there is any error you will return Error. \n    If there is no error, you will return Success.\n    You are only allowed to return 2 words:\n    Error\n    Success\n\n    Input: \ncould not connect to the host as the password as expired.\nInput: \ncould not connect to the host as the password as expired.\n', role='user', context=None)], output_message=CompletionMessage(content='Error', role='assistant', stop_reason='end_of_turn', tool_calls=[]), session_id='ccb6cd50-5f2d-4a92-8b81-6e2654991bd5', started_at=datetime.datetime(2025, 3, 24, 18, 57, 14, 797416, tzinfo=TzInfo(-07:00)), steps=[InferenceStep(api_model_response=CompletionMessage(content='Error', role='assistant', stop_reason='end_of_turn', tool_calls=[]), step_id='44788052-be73-4548-a20a-bfeb9f599a5d', step_type='inference', turn_id='d5876df0-0fef-4c7f-a230-